In [1]:
import numpy as np

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import ExponentialLR

import torchvision

from tqdm.notebook import tqdm

import train as training

In [2]:
dire = "./cleaned-data" # directory of dataset
# loading data loader
trainloader, testloader = training.preprocess_data(directory=dire, batch_size=64, test_size=0.3, rand_num=40, worker=2)

{0: 426, 1: 426}


In [3]:
# load pretrained model
# vgg16 = torchvision.models.vgg16(pretrained=True)
# googlenet = torchvision.models.googlenet(pretrained=True)
# efficientnet_b7 = torchvision.models.efficientnet_b7(pretrained=True)
resnet50 = torchvision.models.resnet50(pretrained=True)

In [4]:
class ClassifierNet(nn.Module):
    def __init__(self, pretrained: nn.Module):
        super().__init__()
        # Pretrained
        self.network = pretrained
        # Replace last layer
        self.network.fc = nn.Sequential(nn.Linear(2048, 512), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.25),
                                         nn.Linear(512, 128), 
                                         nn.ReLU(),  
                                         nn.Dropout(0.50), 
                                         nn.Linear(128, 2),
                                         nn.Softmax()
                                        )
    def forward(self, x):
        out = self.network(x)
        return out


In [5]:
# define params here
model_name = 'resnet50'
net = ClassifierNet(resnet50)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)
scheduler = ExponentialLR(optimizer, gamma=0.9)

criterion = nn.CrossEntropyLoss()
epochs = 50
training.run(model_name, net, optimizer, scheduler, criterion, epochs, trainloader, testloader)

Training on cuda


  0%|          | 0/50 [00:00<?, ?it/s]

C:\Users\goodu\AppData\Local\Programs\Python\Python38\lib\site-packages\torch\nn\modules\container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [1/50], Step [5/10], Accuracy: 48.125, Train Loss: 0.6885
Epoch [1/50], Step [10/10], Accuracy: 51.089, Train Loss: 0.6840
Epoch 1 Learning Rate: 0.009000000000000001
-----------------------------------------------
Epoch [2/50], Step [5/10], Accuracy: 60.625, Train Loss: 0.6894
Epoch [2/50], Step [10/10], Accuracy: 60.637, Train Loss: 0.7009
Epoch 2 Learning Rate: 0.008100000000000001
-----------------------------------------------
Epoch [3/50], Step [5/10], Accuracy: 69.375, Train Loss: 0.6791
Epoch [3/50], Step [10/10], Accuracy: 68.844, Train Loss: 0.6702
Epoch 3 Learning Rate: 0.007290000000000001
-----------------------------------------------
Epoch [4/50], Step [5/10], Accuracy: 76.875, Train Loss: 0.6282
Epoch [4/50], Step [10/10], Accuracy: 74.372, Train Loss: 0.7378
Epoch 4 Learning Rate: 0.006561000000000002
-----------------------------------------------
Epoch [5/50], Step [5/10], Accuracy: 76.562, Train Loss: 0.6060
Epoch [5/50], Step [10/10], Accuracy: 76.717, Train 

KeyboardInterrupt: 